In [1]:
# Some configuations
# This cell defines a magic command to ensure that the script doesn't stop due
# to any error arising in that cell.
from IPython.core.magic import register_cell_magic
@register_cell_magic('handle')
def handle(line, cell):
    try:
#         exec(cell)  # doesn't return the cell output though
        return eval(cell)
    except Exception as exc:
        print(f"\033[1;31m{exc.__class__.__name__} : \033[1;31;47m{exc}\033[0m")
        # raise # if you want the full trace-back in the notebook

# Indexing

In [5]:
import numpy as np
import pandas as pd
pd.read_csv('data/table.csv').head()

,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
0,S_1,C_1,1101,M,street_1,173,63,34.0,A+
1,S_1,C_1,1102,F,street_2,192,73,32.5,B+
2,S_1,C_1,1103,M,street_2,186,82,87.2,B+
3,S_1,C_1,1104,F,street_2,167,81,80.4,B-
4,S_1,C_1,1105,F,street_4,159,64,84.8,B+


In [6]:

df = pd.read_csv('data/table.csv', index_col='ID')
df.head(10)

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+
1201,S_1,C_2,M,street_5,188,68,97.0,A-
1202,S_1,C_2,F,street_4,176,94,63.5,B-
1203,S_1,C_2,M,street_6,160,53,58.8,A+
1204,S_1,C_2,F,street_5,162,63,33.8,B


## (1) [] method

```[]``` method can be used to select column(s) by passing column name(s) (label(s)). Although it also can be used to select row(s), to avoid confusion, we will leave selecting rows to the other two methods, which we will talk about later. 

```df[colname]``` Select single column

In [7]:
df["Address"].head()

ID
1101    street_1
1102    street_2
1103    street_2
1104    street_2
1105    street_4
Name: Address, dtype: object

In [8]:
df[["Address"]].head()

,Address
ID,
1101,street_1
1102,street_2
1103,street_2
1104,street_2
1105,street_4


```df[[col1, col2,..., coln]]``` Select multiple columns

In [9]:
df[["Address", "Math"]].head()

,Address,Math
ID,,
1101,street_1,34.0
1102,street_2,32.5
1103,street_2,87.2
1104,street_2,80.4
1105,street_4,84.8


## (2) loc method

**loc** can be used to index row(s) and column(s) by providing the row and column labels.

```df.loc[row_label(s)]``` Selects single row or subset of rows from the DataFrame by label.

In [10]:
df.loc[1105] # index single row

School          S_1
Class           C_1
Gender            F
Address    street_4
Height          159
Weight           64
Math           84.8
Physics          B+
Name: 1105, dtype: object

In [11]:
df.loc[[1105]] # index single row and keep the dimension

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1105,S_1,C_1,F,street_4,159,64,84.8,B+


In [12]:
df.loc[1105:1203] # index multiple rows

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1105,S_1,C_1,F,street_4,159,64,84.8,B+
1201,S_1,C_2,M,street_5,188,68,97.0,A-
1202,S_1,C_2,F,street_4,176,94,63.5,B-
1203,S_1,C_2,M,street_6,160,53,58.8,A+


In [13]:
%%handle
df.loc[[1105:1203]] # index multiple rows

SyntaxError : invalid syntax (<string>, line 1)


In [14]:
df.loc[[1105, 1102, 1202, 1203]] # index multiple rows

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1105,S_1,C_1,F,street_4,159,64,84.8,B+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1202,S_1,C_2,F,street_4,176,94,63.5,B-
1203,S_1,C_2,M,street_6,160,53,58.8,A+


```df.loc[:, col_labels]``` Selects single column or subset of columns by label

In [15]:
df.loc[:, "School"] # index single column

ID
1101    S_1
1102    S_1
1103    S_1
1104    S_1
1105    S_1
1201    S_1
1202    S_1
1203    S_1
1204    S_1
1205    S_1
1301    S_1
1302    S_1
1303    S_1
1304    S_1
1305    S_1
2101    S_2
2102    S_2
2103    S_2
2104    S_2
2105    S_2
2201    S_2
2202    S_2
2203    S_2
2204    S_2
2205    S_2
2301    S_2
2302    S_2
2303    S_2
2304    S_2
2305    S_2
2401    S_2
2402    S_2
2403    S_2
2404    S_2
2405    S_2
Name: School, dtype: object

In [16]:
df.loc[:, "School":"Weight"].head() # index multiple columns

,School,Class,Gender,Address,Height,Weight
ID,,,,,,
1101,S_1,C_1,M,street_1,173,63
1102,S_1,C_1,F,street_2,192,73
1103,S_1,C_1,M,street_2,186,82
1104,S_1,C_1,F,street_2,167,81
1105,S_1,C_1,F,street_4,159,64


In [17]:
df.loc[:, ["School", "Weight", "Address"]].tail() # index multiple columns

,School,Weight,Address
ID,,,
2401,S_2,62,street_2
2402,S_2,82,street_7
2403,S_2,60,street_6
2404,S_2,84,street_2
2405,S_2,54,street_6


```df.loc[row_label(s), col_label(s)]``` Select both rows and columns by label

In [18]:
df.loc[[1105], ["Address"]]

,Address
ID,
1105,street_4


In [19]:
df.loc[[1105], "Address"]

ID
1105    street_4
Name: Address, dtype: object

In [20]:
df.loc[1105, ["Address"]]

Address    street_4
Name: 1105, dtype: object

In [21]:
df.loc[1105, "Address"]

'street_4'

In [22]:
df.loc[1105:1205, ["Address", "Math"]]

,Address,Math
ID,,
1105,street_4,84.8
1201,street_5,97.0
1202,street_4,63.5
1203,street_6,58.8
1204,street_5,33.8
1205,street_6,68.4


In [23]:
df.index

Index([1101, 1102, 1103, 1104, 1105, 1201, 1202, 1203, 1204, 1205, 1301, 1302,
       1303, 1304, 1305, 2101, 2102, 2103, 2104, 2105, 2201, 2202, 2203, 2204,
       2205, 2301, 2302, 2303, 2304, 2305, 2401, 2402, 2403, 2404, 2405],
      dtype='int64', name='ID')

In [24]:
df.columns

Index(['School', 'Class', 'Gender', 'Address', 'Height', 'Weight', 'Math',
       'Physics'],
      dtype='object')

Index by function

In [25]:
df.head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


In [26]:
df.loc[lambda x:x['Gender']=='M'].head() # here x stands for each row in df

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1201,S_1,C_2,M,street_5,188,68,97.0,A-
1203,S_1,C_2,M,street_6,160,53,58.8,A+
1301,S_1,C_3,M,street_4,161,68,31.5,B+


Index by Boolean Series

In [27]:
df['Address'].head(10)

ID
1101    street_1
1102    street_2
1103    street_2
1104    street_2
1105    street_4
1201    street_5
1202    street_4
1203    street_6
1204    street_5
1205    street_6
Name: Address, dtype: object

In [28]:
df['Address'].isin(['street_7','street_4'])

ID
1101    False
1102    False
1103    False
1104    False
1105     True
1201    False
1202     True
1203    False
1204    False
1205    False
1301     True
1302    False
1303     True
1304    False
1305    False
2101     True
2102    False
2103     True
2104    False
2105     True
2201    False
2202     True
2203     True
2204    False
2205     True
2301     True
2302    False
2303     True
2304    False
2305     True
2401    False
2402     True
2403    False
2404    False
2405    False
Name: Address, dtype: bool

In [29]:
type(df['Address'].isin(['street_7','street_4']))

pandas.core.series.Series

In [30]:
idx = df['Address'].isin(['street_7','street_4'])
df.loc[idx]

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1105,S_1,C_1,F,street_4,159,64,84.8,B+
1202,S_1,C_2,F,street_4,176,94,63.5,B-
1301,S_1,C_3,M,street_4,161,68,31.5,B+
1303,S_1,C_3,M,street_7,188,82,49.7,B
2101,S_2,C_1,M,street_7,174,84,83.3,C
2103,S_2,C_1,M,street_4,157,61,52.5,B-
2105,S_2,C_1,M,street_4,170,81,34.2,A
2202,S_2,C_2,F,street_7,194,77,68.5,B+
2203,S_2,C_2,M,street_4,155,91,73.8,A+


In [31]:
idx = df['Physics'].isin(['B+', 'A', 'A+'])
df.loc[idx]

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1105,S_1,C_1,F,street_4,159,64,84.8,B+
1203,S_1,C_2,M,street_6,160,53,58.8,A+
1301,S_1,C_3,M,street_4,161,68,31.5,B+
1304,S_1,C_3,M,street_2,195,70,85.2,A
2102,S_2,C_1,F,street_6,161,61,50.6,B+
2104,S_2,C_1,F,street_5,159,97,72.2,B+


In [32]:
df.loc[(df["Physics"] == "A+") & (df["Math"] >= 60)] # use "&" (and), "|" (or)  "~" (not) for Pandas

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
2203,S_2,C_2,M,street_4,155,91,73.8,A+


In [33]:
df["Physics"] == "A+"

ID
1101     True
1102    False
1103    False
1104    False
1105    False
1201    False
1202    False
1203     True
1204    False
1205    False
1301    False
1302    False
1303    False
1304    False
1305    False
2101    False
2102    False
2103    False
2104    False
2105    False
2201    False
2202    False
2203     True
2204    False
2205    False
2301    False
2302    False
2303    False
2304    False
2305    False
2401    False
2402    False
2403    False
2404    False
2405    False
Name: Physics, dtype: bool

In [34]:
df["Math"] >= 60

ID
1101    False
1102    False
1103     True
1104     True
1105     True
1201     True
1202     True
1203    False
1204    False
1205     True
1301    False
1302     True
1303    False
1304     True
1305     True
2101     True
2102    False
2103    False
2104     True
2105    False
2201    False
2202     True
2203     True
2204    False
2205     True
2301     True
2302    False
2303     True
2304     True
2305    False
2401    False
2402    False
2403    False
2404     True
2405    False
Name: Math, dtype: bool

In [35]:
(df["Physics"] == "A+") & (df["Math"] >= 60)

ID
1101    False
1102    False
1103    False
1104    False
1105    False
1201    False
1202    False
1203    False
1204    False
1205    False
1301    False
1302    False
1303    False
1304    False
1305    False
2101    False
2102    False
2103    False
2104    False
2105    False
2201    False
2202    False
2203     True
2204    False
2205    False
2301    False
2302    False
2303    False
2304    False
2305    False
2401    False
2402    False
2403    False
2404    False
2405    False
dtype: bool

In [36]:
df.loc[(df["Gender"] == "F") | (df["Height"] >= 170), ["School", "Class"]]

,School,Class
ID,,
1101,S_1,C_1
1102,S_1,C_1
1103,S_1,C_1
1104,S_1,C_1
1105,S_1,C_1
1201,S_1,C_2
1202,S_1,C_2
1204,S_1,C_2
1205,S_1,C_2


## (3) iloc method

**iloc** can be used to index row(s) and column(s) by providing the row and column integer(s).

```df.iloc[row_integer(s)]``` Selects single row or subset of rows from the DataFrame by integer position

**Note**: same as indexing for sequence (but different with ```loc```, it is 0 basis and the selection is close to the left and open to the right (the last item is excluded).

In [37]:
df.iloc[[3]]

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1104,S_1,C_1,F,street_2,167,81,80.4,B-


In [38]:
df.iloc[3]

School          S_1
Class           C_1
Gender            F
Address    street_2
Height          167
Weight           81
Math           80.4
Physics          B-
Name: 1104, dtype: object

In [39]:
df.iloc[:4] # 5th item is not selected

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-


```df.iloc[:, col_integer(s)]``` Selects single column or multiple columns from the DataFrame by integer positions

In [40]:
df.iloc[:, 1:3] # 4th column is not selected

,Class,Gender
ID,,
1101,C_1,M
1102,C_1,F
1103,C_1,M
1104,C_1,F
1105,C_1,F
1201,C_2,M
1202,C_2,F
1203,C_2,M
1204,C_2,F


```df.iloc[row_integer(s), col_integer(s)]``` Selects row and columns from the DataFrame by integer positions

In [41]:
df.iloc[-1::-5, 1:3] 

,Class,Gender
ID,,
2405,C_4,F
2305,C_3,M
2205,C_2,F
2105,C_1,M
1305,C_3,F
1205,C_2,F
1105,C_1,F


In [42]:
list(range(35, 0, -5))

[35, 30, 25, 20, 15, 10, 5]

Index by Boolean **values**

When pass Boolean to `iloc` method, you need to first convert the series as values to `iloc`; it does not take Pandas Series 

In [ ]:
%%handle
df.iloc[(df["Physics"] == "A+") & (df["Math"] >= 60)]

In [ ]:
type((df["Physics"] == "A+"))

In [ ]:
type((df["Physics"] == "A+").values)

In [ ]:
df.iloc[(df["Physics"] == "A+").values]

In [ ]:
df.iloc[(df["Physics"] == "A+").values & (df["Math"] >= 60).values]

In [ ]:
df.iloc[((df["Physics"] == "A+") & (df["Math"] >= 60)).values]

# Data processing

In data analysis, we usually do 

* **split**: Split our data into different groups based on some key(s).
* **apply**: Apply some functions to each group.

![split_apply](figures/split_apply.png)

## split

```goupby``` function can be used to split our data into different groups.

### Group by single column

In [ ]:
 df

In [ ]:
grouped_single = df.groupby('School') # key here is School
grouped_single

In [ ]:
grouped_single.get_group('S_1').head() # get specific group

In [ ]:
grouped_single.get_group('S_2').head() # get specific group

In [ ]:
grouped_single.groups

In [ ]:
grouped_single.size()

In [ ]:
grouped_single.ngroups

In [ ]:
grouped_single.first() # first row of each group

In [ ]:
grouped_single.head() # first five rows of each group

### group by numeric column

In [ ]:
mathgroup = df.groupby(lambda x: "Greater than or equal to 60" if df.loc[x, "Math"]>= 60 else "Less than 60")
mathgroup.get_group("Greater than or equal to 60")

In [ ]:
mathgroup.get_group("Less than 60")

#### ```cut``` method

In [ ]:
bins = [0,40,60,80,90,100]
cuts = pd.cut(df['Math'],bins=bins) 
cuts

In [ ]:
df.groupby(cuts, observed=True).groups

In the above code, the `observed` determines whether the grouping process includes only those combinations of categorical levels that appear in your data (observed combinations) or includes all possible combinations of categories, even if some combinations don't appear in the data.

- **`observed=True`**: Group only by observed combinations (appearing in the dataset).
- **`observed=False`**: Group all possible combinations, including ones that may not appear in the dataset.

In [ ]:
df.groupby(cuts, observed=True).size()

### Group by multiple columns

In [ ]:
grouped_mul = df.groupby(['School','Class'])
grouped_mul.size()

In [ ]:
grouped_mul.groups

In [ ]:
grouped_mul.get_group(('S_2','C_4'))

## Aggregation

Aggregation means the data can be aggregated into a single quantity to describe the data, for example: mean, median, std, etc

### ```apply``` method



In [ ]:
grouped_single['Math'].apply(lambda x: x.sum()) # here, x stands for the data from each group

In [ ]:
grouped_mul['Height'].apply(lambda x: x.max()-x.min()) # range for each group

### ```agg``` method

In [ ]:
df.columns

In [ ]:
grouped_single[['Math', 'Height', 'Weight']].agg(['sum','mean','std'])

In [ ]:
grouped_single.agg({'Height': ['sum','mean','std'], 'Weight': ['median', 'mean', 'max']})

In [ ]:
grouped_single["Math"].agg([('your columna name for math sum','sum'),('math_mean','mean')])

In [ ]:
grouped_mul.agg({'Height': [('Height_mean','mean'),('Height_std','std')],
                 'Weight': [('Weight_median','median'),('Weight_min','min')]})

In [ ]:
grouped_mul.agg({'Height': [('range', lambda x: x.max()-x.min())],
                 'Weight': [('IQR', lambda x: x.quantile(0.75)-x.quantile(0.25))]})

### frequency table by ```pd.crosstab``` method

For categorical variables, the frequency table is often used to summarize the data. You can use ```pd.crosstab``` method to generate frequency tables

In [ ]:
df

In [ ]:
pd.crosstab(index = df.Gender, columns='count')

In [ ]:
pd.crosstab(index = "count", columns=df.Class)

In [ ]:
pd.crosstab(index = df.Gender, columns=df.Class)

In [ ]:
pd.crosstab(index = df.Gender, columns=df.Class, margins=True)

#### relative frequency table

In [ ]:
pd.crosstab(index = df.Gender, columns='count', normalize=True)

In [ ]:
pd.crosstab(index = "count", columns=df.Class, normalize=True)

In [ ]:
pd.crosstab(df.Gender, df.Class, margins=True, normalize=False)

In [ ]:
pd.crosstab(df.Gender, df.Class, margins=True, normalize=True)

# Join

When we have two datesets, we can combine them together

## ```assign``` method

```assign``` method can be used when you want to combine column-wise

In [ ]:
maths = [x+10 for x in df['Math']]
heights = [x+10 for x in df['Weight']]
df.assign(new_math=maths, new_height=heights).head()

In [ ]:
df.head() # no change to original df

In [ ]:
df['new_math2']=maths
df.head()

## ```concat``` method

```pd.concat([df1, df2], axis = 0)``` can be used to combine two dataframe either row-wise or column-wise depends on value of **axis**: 

* 0 (default, row-wise)
* 1 (column-wise).

In [ ]:
df1 = pd.DataFrame({'A': ['A0', 'A1'], 'B': ['B0', 'B1']}, index = [0,1])
df1

In [ ]:
df2 = pd.DataFrame({'A': ['A2', 'A3'], 'B': ['B2', 'B3']}, index = [2,3])
df2

In [ ]:
df3 = pd.DataFrame({'A': ['A1', 'A3'], 'B': ['D1', 'D3']}, index = [1,3])
df3

In [ ]:
pd.concat([df1, df2]) # default axis=0

In [ ]:
a = pd.concat([df1, df3])
a.loc[1]

In [ ]:
pd.concat([df1, df2], axis = 1)

In [ ]:
df4 = pd.concat([df1, df2], axis = 1)
df4['A']

## ```join``` method

The ```join``` method takes two dataframes and joins them on their indexes

In [108]:
# Dataframe of number of sales made by an employee
sales = pd.DataFrame([103, 202, 380, 101, 82, 99, 500],
                     index = ['Tony', 'Sally', 'Randy','Ellen','Fred', 'Tom', 'HanWei'],
                     columns=['sales'])
# Dataframe of all employees and the region they work in
regions = pd.DataFrame(['West', 'South', 'West', 'North', 'South', 'North', 'East', 'West'],
                     index = ['Tony', 'Sally', 'Carl', 'Archie', 'Randy','Ellen','Fred', 'Mo'],
                     columns=['regions'])

In [ ]:
sales

In [ ]:
regions

### left join (default)

Join based on the table on the indeces of the "left" table. Indeces that are not in the left will not be shown

In [ ]:
sales.join(regions)

### right join 

Join based on the table on the indeces of the "right" table. Indeces that are not in the right will not be shown

In [ ]:
sales.join(regions, how='right')

In [ ]:
regions.join(sales)

### inner join 

Only show indeces if they are available in both table (Think this as intersection operation of set)

In [ ]:
regions.join(sales, how='inner')

### outer join 

Show indeces if they they exist in one table (Think this as union operation of set)

In [ ]:
regions.join(sales, how='outer')